<a href="https://colab.research.google.com/github/ZinoTAT/Machine-Learning/blob/master/pytorch/practices/resnet%E6%A8%A1%E5%9E%8B%E5%AE%9E%E7%8E%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
print(torch.__version__)

1.4.0


为方便后续使用，封装一下接下来将要使用的3 * 3卷积和1 * 1卷积

In [0]:
def conv3x3(in_channels, out_channels, stride=1, padding=1):
  return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=padding, bias=False) #bias设置为false是为了在有bn层的情况下节省参数
def conv1x1(in_channels, out_channels, stride=1, padding=0):
  return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=padding, bias=False) #bias设置为false是为了在有bn层的情况下节省参数

![resnet](https://github.com/ZinoTAT/Machine-Learning/blob/master/images/resnet.png?raw=true)

In [0]:
class BasicBlock(nn.Module):
  expansion = 1

  def __init__(self, in_channels, out_channels, stride=1, downsample=None, norm_layer=None):
    super(BasicBlock, self).__init__()
    if norm_layer is None:
      norm_layer = nn.BatchNorm2d
    self.conv1 = conv3x3(in_channels, out_channels, stride)
    self.bn1 = norm_layer(out_channels)
    self.relu = nn.ReLU(inplace=True)
    self.conv2 = conv3x3(out_channels, out_channels, stride)
    self.bn2 = norm_layer(out_channels)
    self.downsample = downsample
    self.stride = stride
  def forward(self, x):
    identity = x

    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)

    x = self.conv2(x)
    x = self.bn2(x)

    if self.downsample is not None:
      identity = self.downsample(identity) #如果输入的x和输出的x存在维度变换，则要先将维度进行统一再相加

    x += identity
    x = self.relu(x)

    return x

In [0]:
class BottleBlock(nn.Module):
  expansion = 4

  def __init__(self, in_channels, out_channels, stride=1, downsample=None, norm_layer=None):
    super(BottleBlock, self).__init__()
    if norm_layer is None:
      norm_layer = nn.BatchNorm2d
    self.conv1 = conv1x1(in_channels, out_channels)
    self.bn1 = norm_layer(out_channels)
    self.conv2 = conv3x3(out_channels, out_channels, stride=stride)
    self.bn2 = norm_layer(out_channels)
    self.conv3 = conv1x1(out_channels, out_channels)
    self.bn3 = norm_layer(out_channels)
    self.relu = nn.ReLU(inplace=True)
    self.downsample = downsample
    self.stride = stride
  def forward(self, x):
    identity = x

    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)

    x = self.conv2(x)
    x = self.bn2(x)
    x = self.relu(x)

    x = self.conv3(x)
    x = self.bn3(x)
    
    if self.downsample is not None:
      identity = self.downsample(identity)
    
    x += identity
    x = self.relu(x)

    return x


In [22]:
def f(x):
  return x.repeat(1, 2, 1, 1)
BB1 = BottleBlock(5, 5)
x = torch.randn([5, 5, 5, 5])
y = BB1(x)
print(y.shape)

torch.Size([5, 5, 5, 5])
